# Exploratory Data Analysis - Customer Loans in Finance

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from load import load_data
from data_transform import DataTransform

## Getting the data and taking a first look

First we get the data.  Data were downloaded from an RDS database using credentials provided by AiCore and saved in a CSV format.
We use the imported function to load the data into a Pandas dataframe.  Then we print its shape followed by information about the data.

In [2]:
df = load_data('initial_data.csv')
print(f"The shape of the dataframe is {df.shape}")
#df.info()

The shape of the dataframe is (54231, 43)


We see that the data consists of 54321 observations (rows) of a set of 43 variables (columns).  We investigate further by printing further information about the dataframe.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54231 entries, 0 to 54230
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           54231 non-null  int64  
 1   member_id                    54231 non-null  int64  
 2   loan_amount                  54231 non-null  int64  
 3   funded_amount                51224 non-null  float64
 4   funded_amount_inv            54231 non-null  float64
 5   term                         49459 non-null  object 
 6   int_rate                     49062 non-null  float64
 7   instalment                   54231 non-null  float64
 8   grade                        54231 non-null  object 
 9   sub_grade                    54231 non-null  object 
 10  employment_length            52113 non-null  object 
 11  home_ownership               54231 non-null  object 
 12  annual_inc                   54231 non-null  float64
 13  verification_sta

Initial observations are that:
- the variables follow a mix of datatypes including 15 objects (strings), 20 floats and 8 integers, making 28 numerical variables
- many variables have a full complement of 54231 observations.  However some variables have missing data and sometimes the gaps are extensive.

## The exploratory analysis - understanding and manipulating the data
Our exploratory analysis will cover five basic stages, undertaken broadly sequentially.  They are
1. Reviewing the appropriateness of datatypes and adjusting as necessary.
2. Handling missing data
3. Considering and handling skewed data
4. Identifying and handling outliers
5. Identifying and handling collinearity among the variables

### Appropriate datatypes

Let us consider each variable in turn.  We can use the pandas value_counts() functionality to get a quick look at each variable, including a few observations, together with the datatype and the count of unique values.  We can hence observe for example:
- whether numbers or text have been assigned inappropriately;
- whether there is a limited number of distinct values (suggesting that the categorical datatype is an option);
- whether data involve times and dates (suggesting that the datetime datatype is an option).

In [4]:
for entry in list(df):
    print(df[entry].value_counts())

id
72323       1
38676116    1
38656203    1
38656154    1
38656128    1
           ..
38656004    1
38656023    1
38656052    1
38656063    1
38656067    1
Name: count, Length: 54231, dtype: int64
member_id
70694       1
41461848    1
41440010    1
41439961    1
41439934    1
           ..
41439809    1
41439828    1
41439857    1
41439868    1
41439872    1
Name: count, Length: 54231, dtype: int64
loan_amount
10000    3951
12000    3062
15000    2844
20000    2625
5000     2013
         ... 
23425       1
31150       1
4775        1
725         1
950         1
Name: count, Length: 1083, dtype: int64
funded_amount
10000.0    3703
12000.0    2863
15000.0    2637
20000.0    2418
5000.0     1913
           ... 
24350.0       1
1850.0        1
24150.0       1
4175.0        1
24750.0       1
Name: count, Length: 1122, dtype: int64
funded_amount_inv
10000.000000    2891
12000.000000    2135
15000.000000    1890
20000.000000    1631
5000.000000     1529
                ... 
1369.990000      

We see that in many cases there are no issues with a variable.  We describe below the exceptions to this rule, and discuss the appropriate actions (index numbers refer to the data info output above).

0   id
Data has been assigned as int64.  While these are numeric characters their purpose is only really to label the data.  We should cast to object to reflect this.

1   member_id
Same as issue as id.  We should cast to object.

5   term
Assigned as object datatype, but in fact only takes two values: 36 months or 60 months.  We could assign as a category.  We may also want to strip the text "months" in which case we could cast to numeric.  One advantage here is that we could impute the missing data using the mean.  This decision may need to be revisited when we have a better idea of what role this variable plays in our analysis.  Initially we will strip months, and cast to float.

8   grade
Assigned as object.  In fact, it shows which of 7 grades a loan is categorised by.  We will hence assign as a categorical datatype.
We also need to bear in mind that there is an ordinal relation between the categories and consider whether this will be useful.  One option might be to convert letters to numbers, thus allowing calculating of "average grades."  To be kept under consideration as the analysis proceeds.

9   subgrade
Same issues as grade, albeit with more categories.  
Again, we will cast to category and keep under review how to use the information stored in the "ordinal nature" of this variable.

10  employment_length
Describes employment length in blocks of years up to "10+".  We will convert to category.
We may want to strip years and treat as numerical variable and will note this possibility at this stage.

11  home_ownership
Assigned as object with 3 main populated categories, plus a few "Other" and "None".  We will cast to category and consider later whether to keep or handle Other/None.

13  verification_status
Assigned as object with 3 categories,   We will cast to category.

14  issue_date
Assigned as object but actually represents dates in month/year format.  It might be useful to cast to datetime.  As date objects minimally require day, month, year we will need to assign an arbitrary day value to these observations.  We will add "day 15" and cast to datetime datatype.

15  loan_status
We can cast to category

16  payment_plan
We could cast to category but with a 54230 - 1 split its doubtful that this data will be useful in the analysis anyway.  We will drop.

17  purpose
Assigned as object but actually divides into 14 "pure" categories.   We will cast to category.

20  earliest_credit_line
As issue date.  We will add "day 15" and cast to datetime datatype.

21  inq_last_6_months
Fits into 14 numeric categories.  We can leave as numeric.

35  last_payment_date
As issue date.  We will add "day 15" and cast to datetime datatype.

37  next_payment_date
As issue date.  We will add "day 15" and cast to datetime datatype.

38  last_credit_pull_date
As issue date.  We will add "day 15" and cast to datetime datatype.

41  policy_code
All coded as "1".  It informs us about how we've drawn the sample, but won't be of further use.  Drop.

42  application_type
All are "Individual".  It informs us about how we've drawn the sample, but won't be of further use.  Drop.

We use methods in our DataTransform class to perform these transformations.

First we cast the first two columns to object.

In [5]:
transformer = DataTransform(df)

#transformer.dataframe.info()

target_cols = ['id' , 'member_id']
for entry in target_cols:
    transformer.col_to_object(entry)

#transformer.dataframe.info()

Next we cast seven columns to category.

In [6]:
transformer.dataframe.info()

target_cols = ['grade', 'sub_grade', 'employment_length', 'home_ownership', 'verification_status', 'loan_status', 'purpose']
for entry in target_cols:
    transformer.col_to_category(entry)

transformer.dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54231 entries, 0 to 54230
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           54231 non-null  object 
 1   member_id                    54231 non-null  object 
 2   loan_amount                  54231 non-null  int64  
 3   funded_amount                51224 non-null  float64
 4   funded_amount_inv            54231 non-null  float64
 5   term                         49459 non-null  object 
 6   int_rate                     49062 non-null  float64
 7   instalment                   54231 non-null  float64
 8   grade                        54231 non-null  object 
 9   sub_grade                    54231 non-null  object 
 10  employment_length            52113 non-null  object 
 11  home_ownership               54231 non-null  object 
 12  annual_inc                   54231 non-null  float64
 13  verification_sta

We address the column "term", by stripping the months part of the text, then casting to float

In [7]:
transformer.strip_months('term')
transformer.col_to_float('term')

transformer.dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54231 entries, 0 to 54230
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   id                           54231 non-null  object  
 1   member_id                    54231 non-null  object  
 2   loan_amount                  54231 non-null  int64   
 3   funded_amount                51224 non-null  float64 
 4   funded_amount_inv            54231 non-null  float64 
 5   term                         49459 non-null  float64 
 6   int_rate                     49062 non-null  float64 
 7   instalment                   54231 non-null  float64 
 8   grade                        54231 non-null  category
 9   sub_grade                    54231 non-null  category
 10  employment_length            52113 non-null  category
 11  home_ownership               54231 non-null  category
 12  annual_inc                   54231 non-null  float64 
 13  v

We convert the columns to time, provided there are no null values.  

The following columns have nulls: 'last_payment_date', 'next_payment_date', 'last_credit_pull_date'
We'll consider handling these nulls later, and if they're dropped we will return to this point.

In [8]:
target_cols = ['issue_date', 'earliest_credit_line']
for entry in target_cols:
    transformer.add_day_15(entry)
    transformer.col_to_date(entry)

transformer.dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54231 entries, 0 to 54230
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   id                           54231 non-null  object        
 1   member_id                    54231 non-null  object        
 2   loan_amount                  54231 non-null  int64         
 3   funded_amount                51224 non-null  float64       
 4   funded_amount_inv            54231 non-null  float64       
 5   term                         49459 non-null  float64       
 6   int_rate                     49062 non-null  float64       
 7   instalment                   54231 non-null  float64       
 8   grade                        54231 non-null  category      
 9   sub_grade                    54231 non-null  category      
 10  employment_length            52113 non-null  category      
 11  home_ownership               54231 non-nu

Finally, we drop the three columns identified.

In [9]:
target_cols = ['policy_code', 'application_type', 'payment_plan']
for entry in target_cols:
    transformer.drop_col(entry)

In [10]:
df = transformer.dataframe
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54231 entries, 0 to 54230
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   id                           54231 non-null  object        
 1   member_id                    54231 non-null  object        
 2   loan_amount                  54231 non-null  int64         
 3   funded_amount                51224 non-null  float64       
 4   funded_amount_inv            54231 non-null  float64       
 5   term                         49459 non-null  float64       
 6   int_rate                     49062 non-null  float64       
 7   instalment                   54231 non-null  float64       
 8   grade                        54231 non-null  category      
 9   sub_grade                    54231 non-null  category      
 10  employment_length            52113 non-null  category      
 11  home_ownership               54231 non-nu